In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders import BinaryEncoder

In [2]:
train_identity = pd.read_csv('train_identity.csv',index_col='TransactionID')
train_transaction = pd.read_csv('train_transaction.csv',index_col='TransactionID')

In [3]:
train_transaction.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
y = train_transaction['isFraud'].copy()
train_transaction.drop('isFraud',axis=1,inplace=True)

In [5]:
train_identity.head()

,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [6]:
#checking the target variable distribution
100*y.value_counts()/y.shape[0]

0    96.500999
1     3.499001
Name: isFraud, dtype: float64

In [7]:
train_identity.isna().mean().sort_values(ascending=True)*100

id_01          0.000000
id_12          0.000000
id_38          2.251912
id_37          2.251912
id_36          2.251912
id_35          2.251912
id_15          2.251912
id_29          2.256765
id_28          2.256765
id_11          2.256765
id_02          2.330257
DeviceType     2.373243
id_31          2.739318
id_17          3.372321
id_19          3.407681
id_20          3.447200
id_05          5.108401
id_06          5.108401
id_16         10.325654
id_13         11.726165
DeviceInfo    17.726179
id_14         44.503685
id_34         46.056034
id_32         46.207872
id_30         46.222432
id_09         48.052110
id_10         48.052110
id_33         49.187079
id_03         54.016071
id_04         54.016071
id_18         68.722137
id_23         96.416215
id_27         96.416215
id_22         96.416215
id_26         96.420375
id_21         96.423149
id_08         96.425922
id_07         96.425922
id_25         96.441868
id_24         96.708798
dtype: float64

In [8]:
train_transaction.isna().mean().sort_values(ascending=True)*100

TransactionDT      0.000000
C14                0.000000
C13                0.000000
C12                0.000000
C11                0.000000
C10                0.000000
C9                 0.000000
C8                 0.000000
C7                 0.000000
C6                 0.000000
C5                 0.000000
C4                 0.000000
C3                 0.000000
C1                 0.000000
C2                 0.000000
TransactionAmt     0.000000
ProductCD          0.000000
card1              0.000000
V309               0.002032
V306               0.002032
V307               0.002032
V299               0.002032
V321               0.002032
V320               0.002032
V319               0.002032
V317               0.002032
V316               0.002032
V312               0.002032
V311               0.002032
V310               0.002032
                    ...    
V143              86.122701
V159              86.122701
V160              86.122701
V164              86.122701
V156              86

In [9]:
categorical_columns_transaction = ['ProductCD','card1','card2','card3','card4','card5','card6','addr1', 'addr2','P_emaildomain',
                                   'R_emaildomain','M1','M2','M3','M4','M5','M6','M7','M8','M9']
numerical_columns_transaction = [i for i in train_transaction if i not in categorical_columns_transaction]

In [10]:
categorical_columns_identity = ['DeviceType','DeviceInfo'] + [i for i in train_identity.columns if "id" in i and int(i.split("_")[1]) >=12]
numerical_columns_identity = [i for i in train_identity.columns if i not in categorical_columns_identity]

In [11]:
def labelencoding(dataset,columns_list,mapping):
    dataset.replace(dict.fromkeys(columns_list,mapping),inplace=True)

### Labelling categorical columns from transaction table

In [12]:
columns_list = ['M1','M2','M3','M5','M6','M7','M8','M9']
mapping = {'T':1, 'F':0}
labelencoding(train_transaction,columns_list,mapping)
labelencoding(train_transaction,['M4'],{'M2':2,'M1':1,'M0':0})
enc = BinaryEncoder(cols=['ProductCD','card4','card6','P_emaildomain','R_emaildomain']).fit(train_transaction)
train_transaction = enc.transform(train_transaction)

In [32]:
categorical_columns_transaction = [i for i in train_transaction if i not in numerical_columns_transaction]

In [35]:
train_transaction[categorical_columns_transaction] = train_transaction[categorical_columns_transaction].astype('category')

### Labelling categorical columns from identity table

In [ ]:
#coverting True False to 1,0
['id_35','id_36','id_37','id_38']
#converting FOund, NotFound to 1,0
['id_12','id_15','id_27','id_28','id_29']

In [ ]:
obj = LabelBinarizer()
obj.fit_transform(train_identity[['id_35','id_36','id_37','id_38']])

In [ ]:
train_identity[['id_35','id_36','id_37','id_38']].nunique()

In [ ]:
for i in ['ProductCD']:
    plt.figsize=(13,7)
    sns.boxplot(y=train_transaction[i],x=y)
    plt.show()

In [ ]:
dict.fromkeys(categorical_columns_identity,None)